In [1]:
import requests
import duckdb
import pandas as pd
import os
from datetime import datetime
import logging
from concurrent.futures import ThreadPoolExecutor

In [4]:
# ! pip install --upgrade pip setuptools wheel
# ! pip install requests

In [25]:
API_URL = "http://localhost:8000/v1/patients"
response = requests.get(
        API_URL,
        params={
            "offset": 0,
            "limit": 100,
            "sort_field": "last_name",
            "sort_dir": "asc",
        }
    )
response.raise_for_status() 

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /v1/patients?offset=0&limit=100&sort_field=last_name&sort_dir=asc (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x120ac9db0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [33]:
batch_data=response.json()['data']

In [25]:
# wk_dir=os.getcwd()
# print(wk_dir)
DB_PATH="destination.duckdb"

In [45]:
conn = duckdb.connect(DB_PATH)
# connection = duckdb.connect()

In [73]:
conn.execute("DROP TABLE IF EXISTS patients")
conn.execute("""
            CREATE TABLE patients (
                id INTEGER primary key,
                first_name VARCHAR,
                last_name VARCHAR,
                email VARCHAR,
                date_of_birth VARCHAR,
                created_at VARCHAR,
                updated_at VARCHAR,
                total_visits INTEGER,
                timestamp TIMESTAMP
            )
        """)
conn.execute("DESCRIBE patients").fetchdf()

,column_name,column_type,null,key,default,extra
0,id,INTEGER,NO,PRI,None,None
1,first_name,VARCHAR,YES,None,None,None
2,last_name,VARCHAR,YES,None,None,None
3,email,VARCHAR,YES,None,None,None
4,date_of_birth,VARCHAR,YES,None,None,None
5,created_at,VARCHAR,YES,None,None,None
6,updated_at,VARCHAR,YES,None,None,None
7,total_visits,INTEGER,YES,None,None,None
8,timestamp,TIMESTAMP,YES,None,None,None


In [54]:
df = pd.DataFrame(batch_data)
df.head()
df['timestamp'] = datetime.now()
conn.register("patients_temp", df)

In [101]:
conn.execute(query).fetchdf()

,table_name
0,patients
1,patients_temp2
2,patients_temp


In [100]:
# df.to_sql('patients_temp2', conn, if_exists='replace', index=False)

/var/folders/3j/5_lv34mn0rj6phj00mbp9sgm0000gp/T/ipykernel_57895/4123938316.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('patients_temp2', conn, if_exists='replace', index=False)


-1

In [67]:
conn.execute("DESCRIBE patients_temp").fetchdf()

,column_name,column_type,null,key,default,extra
0,email,VARCHAR,YES,None,None,None
1,first_name,VARCHAR,YES,None,None,None
2,last_name,VARCHAR,YES,None,None,None
3,date_of_birth,VARCHAR,YES,None,None,None
4,id,BIGINT,YES,None,None,None
5,created_at,VARCHAR,YES,None,None,None
6,updated_at,VARCHAR,YES,None,None,None
7,total_visits,BIGINT,YES,None,None,None
8,timestamp,TIMESTAMP,YES,None,None,None


In [76]:
conn.execute("select * from patients limit 10").fetchdf()

,id,first_name,last_name,email,date_of_birth,created_at,updated_at,total_visits,timestamp
0,236,Jennifer,Adams,peter82@johnson.com,2018-06-02,2024-08-19T15:13:20,2024-10-04T20:32:50,0,2024-12-26 23:16:41.000349
1,57,Brian,Adkins,hbenton@hubbard.net,2009-09-01,2021-06-08T18:21:30,2023-09-30T23:13:24,0,2024-12-26 23:16:41.000349
2,201,Kelly,Aguirre,gomezcatherine@smith-sullivan.com,2008-02-16,2024-10-14T05:22:04,2024-11-05T23:50:12,0,2024-12-26 23:16:41.000349
3,128,Nicholas,Alexander,wross@gmail.com,1939-07-29,2023-09-25T18:13:06,2024-10-14T15:14:25,0,2024-12-26 23:16:41.000349
4,159,Rebecca,Alexander,blacktyler@francis-hall.com,1975-11-02,2020-02-01T16:20:07,2023-02-15T11:50:02,0,2024-12-26 23:16:41.000349
5,38,Stefanie,Allen,kpayne@baker.com,2022-06-03,2020-05-29T00:04:02,2023-12-24T09:29:35,0,2024-12-26 23:16:41.000349
6,93,Christopher,Allen,michaelmartinez@gibson.net,2000-06-01,2023-04-25T14:50:54,2024-09-24T16:55:50,0,2024-12-26 23:16:41.000349
7,117,Melissa,Allen,deborah80@yahoo.com,1927-09-23,2023-11-19T19:14:03,2023-12-14T16:08:18,0,2024-12-26 23:16:41.000349
8,219,Jo,Allen,ryanjackson@clark.com,1972-08-22,2021-11-12T07:35:38,2022-11-09T23:33:00,0,2024-12-26 23:16:41.000349
9,537,Yvonne,Allen,brianna60@gmail.com,2011-03-04,2022-02-19T06:34:18,2022-07-01T12:18:02,0,2024-12-26 23:16:41.000349


In [75]:
ins = """
    INSERT INTO patients
    SELECT id,first_name,last_name,email,date_of_birth,
    created_at,updated_at,total_visits,timestamp 
    FROM patients_temp
    ON CONFLICT (id) DO UPDATE
            SET first_name = excluded.first_name,
                last_name = excluded.last_name,
                email = excluded.email,
                updated_at = excluded.updated_at,
                total_visits = excluded.total_visits,
                timestamp = excluded.timestamp;
"""
x=conn.execute(ins)

In [31]:
connect.close()

In [32]:
connect = duckdb.connect("data/destination.duckdb")

In [33]:
# connect.execute("select * from patients_temp limit 10").fetchdf()
query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
"""
connect.execute(query).fetchdf()

,table_name
0,patients


In [34]:
connect.execute("select * from patients limit 10").fetchdf()
# connect.execute("Drop Table patients").fetchdf()

,id,first_name,last_name,email,date_of_birth,created_at,updated_at,total_visits,timestamp
0,236,Jennifer,Adams,peter82@johnson.com,2018-06-02,2024-08-19T15:13:20,2024-10-04T20:32:50,0,2024-12-27 10:38:27.099394
1,57,Brian,Adkins,hbenton@hubbard.net,2009-09-01,2021-06-08T18:21:30,2023-09-30T23:13:24,0,2024-12-27 10:38:27.099394
2,201,Kelly,Aguirre,gomezcatherine@smith-sullivan.com,2008-02-16,2024-10-14T05:22:04,2024-11-05T23:50:12,0,2024-12-27 10:38:27.099394
3,128,Nicholas,Alexander,wross@gmail.com,1939-07-29,2023-09-25T18:13:06,2024-10-14T15:14:25,0,2024-12-27 10:38:27.099394
4,159,Rebecca,Alexander,blacktyler@francis-hall.com,1975-11-02,2020-02-01T16:20:07,2023-02-15T11:50:02,0,2024-12-27 10:38:27.099394
5,38,Stefanie,Allen,kpayne@baker.com,2022-06-03,2020-05-29T00:04:02,2023-12-24T09:29:35,0,2024-12-27 10:38:27.099394
6,93,Christopher,Allen,michaelmartinez@gibson.net,2000-06-01,2023-04-25T14:50:54,2024-09-24T16:55:50,0,2024-12-27 10:38:27.099394
7,117,Melissa,Allen,deborah80@yahoo.com,1927-09-23,2023-11-19T19:14:03,2023-12-14T16:08:18,0,2024-12-27 10:38:27.099394
8,219,Jo,Allen,ryanjackson@clark.com,1972-08-22,2021-11-12T07:35:38,2022-11-09T23:33:00,0,2024-12-27 10:38:27.099394
9,537,Yvonne,Allen,brianna60@gmail.com,2011-03-04,2022-02-19T06:34:18,2022-07-01T12:18:02,0,2024-12-27 10:38:27.099394
